In [35]:

#file_path = "config.txt"  # Ajuste o caminho para o arquivo de configuração

            # Get the current working directory
#current_dir = os.getcwd()

        #with open(file_path, 'r') as config_file:
#    lines = config_file.readlines()
#    origem = os.path.join(current_dir, lines[0].strip().split('=')[1])
#    destino = os.path.join(current_dir, lines[1].strip().split('=')[1])
#    print(f"Origem é {origem} e destino é {destino}")
#    print(os.path.join(current_dir)

In [48]:

import os
import shutil
import re

from PyPDF2 import PdfReader

import locale
from datetime import datetime

# Configura o idioma para português (Brasil)- faz teste  para saber se o sistema operacional suporta a configuração
try:
    locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')  # Para Linux/MacOS
except locale.Error:
    try:
        locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')  # Para Windows
    except locale.Error:
        print("Configuração de localidade não suportada. Usando padrão do sistema.")



    #****** inicio - acrescentado para pegar a data de extração do extrato

def extract_period_extrato(pdf_path):
    """
    Lê o PDF e extrai a informação de período na primeira página
    a partir do literal 'Período do extrato'.

    Args:
        pdf_path (str): Caminho para o arquivo PDF.

    Returns:
        str: Mês e ano formatados como 'MESYY', ou None se não encontrar os padrões.
    """
    try:
        # Abrir o PDF no modo de leitura binária
        with open(pdf_path, 'rb') as pdf_file:
            reader = PdfReader(pdf_file)
            # Extrair o texto da primeira página
            first_page_content = reader.pages[0].extract_text()

        # Procurar a linha com "Período do extrato"
        match = re.search(r"Período do extrato.*?((\d{2})\s*/\s*(\d{4})|Mês atual)", first_page_content, re.IGNORECASE)
        
        if not match:
            print("Nenhum padrão correspondente foi encontrado.")
            return None

        # Identificar o padrão encontrado
        if match.group(1).lower() == "mês atual":
            # Usar mês e ano atuais
            now = datetime.now()
            return now.strftime('%b').upper()[:3] + now.strftime('%y')
        else:
            # Extrair o mês e ano do padrão "11 / 2024"
            month, year = int(match.group(2)), int(match.group(3))
            date_obj = datetime(year, month, 1)
            return date_obj.strftime('%b').upper()[:3] + str(year)[-2:]
    except Exception as e:
        print(f"Erro ao processar o PDF: {e}")
        return None


    #******* fim - acrescentado para pegar a data de extração do extrato


def read_config(file_path):
    with open(file_path, 'r') as config_file:
        lines = config_file.readlines()
        origem = lines[0].strip().split('=')[1]
        destino = lines[1].strip().split('=')[1]
    return origem, destino

def get_month_abbreviation(date_str):
    months = ["JAN", "FEV", "MAR", "ABR", "MAI", "JUN", "JUL", "AGO", "SET", "OUT", "NOV", "DEZ"]
    date_obj = datetime.strptime(date_str, "%Y%m%d")
    return months[date_obj.month - 1] + str(date_obj.year)[2:]

def move_file(file_path, destino, reason=""):
    """Move um arquivo para o destino com uma mensagem de motivo."""
    file_name = os.path.basename(file_path)
    new_path = os.path.join(destino, file_name)
    print(f"Movendo {file_name} para {destino}. Motivo: {reason}")
    shutil.move(file_path, new_path)

def process_ofx(file_path, destino):
    file_name = os.path.basename(file_path)
    if "Bradesco_" in file_name:
        move_file(file_path, destino, "Arquivo identificado como 'Bradesco_'")
        return

    print(f"Processando arquivo OFX: {file_path}")
    with open(file_path, 'r') as file:
        content = file.read()
    dtend_match = re.search(r"<DTEND>(\d{8})", content)
    if not dtend_match:
        print(f"Tag <DTEND> não encontrada em {file_path}")
        return
    dtend = dtend_match.group(1)
    month_year = get_month_abbreviation(dtend)

    prefixes = {
        "2645348341": "imp - CARROSSEL - Extrato2645348341 - ",
        "2645378186": "imp - AIR - Extrato2645378186 - ",
        "2645376434": "imp - Mouta - Extrato2645376434 - ",
        "2645376442": "imp - Segurança - Extrato2645376442 - ",
        "36498535478": "imp - Segurança - Pará - Extrato36498535478 - ",
    }
    for key, prefix in prefixes.items():
        if key in file_name:
            new_name = f"{prefix}{month_year}.ofx"
            print(f"Renomeando {file_name} para {new_name}")
            shutil.move(file_path, os.path.join(destino, new_name))
            return
    print(f"Nenhum prefixo correspondente para {file_name}")

def process_pdf(file_path, destino):
    file_name = os.path.basename(file_path)
    if "Bradesco_" in file_name:
        move_file(file_path, destino, "Arquivo identificado como 'Bradesco_'")
        return

    print(f"Processando arquivo PDF: {file_path}")
    try:
        reader = PdfReader(file_path)
        first_page_content = reader.pages[0].extract_text()  # Extraindo o conteúdo da primeira página
    except Exception as e:
        print(f"Erro ao processar o PDF {file_path}: {e}")
        return

    # Verifica se é extrato de conta corrente
    if "Extrato de conta corrente" in first_page_content:
        print("Encontrado 'Extrato de conta corrente' no conteúdo.")
        account_match = re.search(r"Conta corrente (\d{5}-\d)", first_page_content)
        if not account_match:
            print("Número da conta não encontrado no PDF.")
            return
        account = account_match.group(1)
        prefixes = {
            "53547-8": "SEGURANÇA - Pará",
            "37644-2": "SEGURANÇA",
            "37643-4": "MOUTA",
            "37818-6": "AIR",
            "34834-1": "CARROSSEL",
        }
        prefix = prefixes.get(account, None)
        if prefix:
            #month_year = datetime.now().strftime("%b%y").upper()
            month_year = extract_period_extrato(file_path)
            new_name = f"Banco do Brasil - {prefix} - Extrato - {month_year}.pdf"
            print(f"Renomeando e movendo {file_path} para {new_name}")
            shutil.move(file_path, os.path.join(destino, new_name))

    # Verifica se é comprovante de pagamento ou transferência
    elif "COMPROVANTE DE PAGAMENTO DE TITULOS" in first_page_content or "COMPROVANTE DE TRANSFERENCIA" in first_page_content or "Comprovante Pix" in first_page_content:
        tipo_comprovante = "Comprovantes"
        print(f"Encontrado '{tipo_comprovante}' no conteúdo.")
        
        padrao_conta = r'\b\d{2}\.\d{3}-\d\b'
        account_match = re.search(padrao_conta, first_page_content)
        if not account_match:
            print("Número da conta não encontrado no PDF.")
            return
        account = account_match.group(0)

        #---código acrescentado inicio
        match = re.search(r"DATA.*?(\d{2}/\d{2}/\d{4})", first_page_content)
        if not match:
            # Segundo padrão: 'DEBITO EM: dd/mm/yyyy'
            match = re.search(r"DEBITO EM:.*?(\d{2}/\d{2}/\d{4})", first_page_content, re.IGNORECASE)
        
        if match:
            # Extrai a data encontrada
            date_str = match.group(1)
            
            try:
                # Converte a data para um objeto datetime
                date_obj = datetime.strptime(date_str, '%d/%m/%Y')
                
                # Retorna o mês abreviado (por exemplo, NOV para novembro)
                month_year = date_obj.strftime('%b%y').upper()
            except ValueError:
                # Erro de formatação de data
                print(f"Erro ao interpretar a data: {date_str}")
                return None
        else:
            print("Nenhuma data encontrada no formato DD/MM/YYYY após 'DATA'.")
            return None

        
        #---código acrescentado fim

        
        prefixes = {
            "53.547-8": "SEGURANÇA - Pará",
            "37.644-2": "SEGURANÇA",
            "37.643-4": "MOUTA",
            "37.818-6": "AIR",
            "34.834-1": "CARROSSEL",
        }
        prefix = prefixes.get(account, None)
        if prefix:
            #month_year = datetime.now().strftime("%b%y").upper()
            new_name = f"Banco do Brasil - {prefix} - {tipo_comprovante} - {month_year}.pdf"
            print(f"Renomeando e movendo {file_path} para {new_name}")
            shutil.move(file_path, os.path.join(destino, new_name))
        else:
            print(f"Prefixo não encontrado para a conta {account}")

def main():
    config_path = "config.txt"  # Ajuste o caminho para o arquivo de configuração
    origem, destino = read_config(config_path)

    for file_name in os.listdir(origem):
        file_path = os.path.join(origem, file_name)
        if file_name.lower().endswith('.ofx'):
            process_ofx(file_path, destino)
        elif file_name.lower().endswith('.pdf'):
            process_pdf(file_path, destino)
        else:
            print(f"Arquivo ignorado: {file_name}")

if __name__ == "__main__":
    main()



Arquivo ignorado: BOLETOS
Arquivo ignorado: EXTRATOS
Arquivo ignorado: Renomeado
Arquivo ignorado: Todos
Processando arquivo PDF: G:/.shortcut-targets-by-id/1qzatcP0IbuDDiwYUI1hx9AY_kjft8mRB/EXTRATOS\CARROSSEL - - DEZ2024.pdf
Encontrado 'Comprovantes' no conteúdo.
Renomeando e movendo G:/.shortcut-targets-by-id/1qzatcP0IbuDDiwYUI1hx9AY_kjft8mRB/EXTRATOS\CARROSSEL - - DEZ2024.pdf para Banco do Brasil - CARROSSEL - Comprovantes - DEZ24.pdf
Processando arquivo PDF: G:/.shortcut-targets-by-id/1qzatcP0IbuDDiwYUI1hx9AY_kjft8mRB/EXTRATOS\AIR - - DEZ2024.pdf
Encontrado 'Comprovantes' no conteúdo.
Renomeando e movendo G:/.shortcut-targets-by-id/1qzatcP0IbuDDiwYUI1hx9AY_kjft8mRB/EXTRATOS\AIR - - DEZ2024.pdf para Banco do Brasil - AIR - Comprovantes - DEZ24.pdf
Processando arquivo PDF: G:/.shortcut-targets-by-id/1qzatcP0IbuDDiwYUI1hx9AY_kjft8mRB/EXTRATOS\MOUTA - - DEZ2024.pdf
Encontrado 'Comprovantes' no conteúdo.
Renomeando e movendo G:/.shortcut-targets-by-id/1qzatcP0IbuDDiwYUI1hx9AY_kjft8mRB/E

comando abaixo para gerar o arquivo py para ser compilado

In [49]:
!jupyter nbconvert --to script TratarExtCompOfx.ipynb

[NbConvertApp] Converting notebook TratarExtCompOfx.ipynb to script
[NbConvertApp] Writing 9437 bytes to TratarExtCompOfx.py


Código abaixo para compilar e gerar o executável dentro do Command do Windows

In [ ]:
pyinstaller --onefile TratarExtCompOfx.py

In [ ]:
#pip install pymupdf